In [1]:
import requests
import pandas as pd

# Function to fetch FDI data from World Bank API
def fetch_fdi_data_from_world_bank(country_code='all', indicator='BX.KLT.DINV.CD.WD', format='json', per_page=1000):
    base_url = "https://api.worldbank.org/v2/country"
    url = f"{base_url}/{country_code}/indicator/{indicator}?format={format}&per_page={per_page}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            return data[1]  # The first element is general metadata, the second contains the data
        else:
            return None
    else:
        return None

# Fetch FDI data for all countries
fdi_data_json = fetch_fdi_data_from_world_bank()

# Convert the retrieved JSON data into a DataFrame
if fdi_data_json:
    fdi_data_df = pd.json_normalize(fdi_data_json)
    print(fdi_data_df.head())
else:
    print("Failed to retrieve FDI data from World Bank API.")


  countryiso3code  date         value unit obs_status  decimal  \
0             AFE  2023  2.111425e+10                        0   
1             AFE  2022  2.065036e+10                        0   
2             AFE  2021  5.441385e+10                        0   
3             AFE  2020  1.365273e+10                        0   
4             AFE  2019  1.492730e+10                        0   

        indicator.id                                    indicator.value  \
0  BX.KLT.DINV.CD.WD  Foreign direct investment, net inflows (BoP, c...   
1  BX.KLT.DINV.CD.WD  Foreign direct investment, net inflows (BoP, c...   
2  BX.KLT.DINV.CD.WD  Foreign direct investment, net inflows (BoP, c...   
3  BX.KLT.DINV.CD.WD  Foreign direct investment, net inflows (BoP, c...   
4  BX.KLT.DINV.CD.WD  Foreign direct investment, net inflows (BoP, c...   

  country.id                country.value  
0         ZH  Africa Eastern and Southern  
1         ZH  Africa Eastern and Southern  
2         ZH  Africa

In [ ]:
#please ignor for now
import requests
import pandas as pd

# Define the API URL for UN Comtrade
base_url = "https://comtrade.un.org/api/get"

# Parameters for API call
params = {
    'type': 'C',  # Commodity
    'freq': 'A',  # Annual data
    'px': 'HS',  # Harmonized System classification
    'r': '566',  # Reporter code for Nigeria (as an example)
    'ps': '2019,2020,2021,2022,',  # Period (years)
    'p': 'all',  # Partner country (all countries)
    'rg': 'all',  # Trade flow (imports/exports)
    'cc': 'TOTAL',  # Commodity code (all commodities)
    'fmt': 'json'  # Response format
}

# Make the request
response = requests.get(base_url, params=params)
data = response.json()

# Parse and convert to DataFrame
trade_data = pd.json_normalize(data['dataset'])
print(trade_data.head())


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# 

In [ ]:
# extract data from the API
import requests
import pandas as pd

# List of West African states
countries = [
    'BEN', 'BFA', 'CPV', 'CIV', 'GMB', 
    'GHA', 'GIN', 'GNB', 'LBR', 'MLI', 
    'MRT', 'NER', 'NGA', 'SEN', 'SLE', 'TGO'
]

# Indicators for different metrics
indicators = {
    'Ease of Doing Business': 'IC.BUS.EASE.XQ',
    'Starting a Business': 'IC.REG.DURS',
    'Getting Credit': 'IC.LGL.CRED.XQ',
    'Trading Across Borders': 'IC.EXP.COST.XQ',
    'Protecting Minority Investors': 'IC.INV.MINR',
    'GDP Growth Rate': 'NY.GDP.MKTP.KD.ZG',
    'GDP per Capita': 'NY.GDP.PCAP.CD',
    'GDP By Sector': 'NY.GDP.MKTP.CD',
    'Inflation Rate': 'FP.CPI.TOTL.ZG',
    'Population Size': 'SP.POP.TOTL',
    'Urban Population': 'SP.URB.TOTL.IN.ZS',
    'Labor Force Participation Rate': 'SL.TLF.CACT.ZS',
    'Youth Unemployment Rate': 'SL.UEM.1524.ZS',
    'Access to Electricity': 'EG.ELC.ACCS.ZS',
    'Internet Penetration': 'IT.NET.USER.ZS',
    'Tariff Rates': 'TM.TAX.MRCH.WM.AR.ZS',
    'Foreign Direct Investment (FDI) Inflows': 'BX.KLT.DINV.WD.GD.ZS',
    'Political Stability Index': 'PV.POL.STAB',
    'Control of Corruption': 'CC.EST',
    'Regulatory Quality': 'RQ.EST',
    'Health Expenditure per Capita': 'SH.XPD.CHEX.PC.CD',
    'Education Expenditure (% of GDP)': 'SE.XPD.TOTL.GD.ZS',
    'Poverty Headcount Ratio': 'SI.POV.DDAY',
    'Gini Index': 'SI.POV.GINI'
}

# Base URL for the World Bank API
base_url = "http://api.worldbank.org/v2/country/{}/indicator/{}?date={}&format=json"

# Year range of interest (you can modify this as needed)
year_range = "2019:2023"

# List to store collected data
data_results = []

# Loop through each country and extract data for each indicator
for country in countries:
    for indicator_name, indicator_code in indicators.items():
        url = base_url.format(country, indicator_code, year_range)
        response = requests.get(url)
        response_data = response.json()

        # Check if the response is in the expected format
        if isinstance(response_data, list) and len(response_data) > 1 and isinstance(response_data[1], list):
            if len(response_data[1]) > 0:
                for data_entry in response_data[1]:
                    # Collect data if it's available
                    year = data_entry.get('date')
                    value = data_entry.get('value')
                    if value is not None:  # Only add data if there is a valid value
                        data_results.append({
                            'Country': country,
                            'Indicator': indicator_name,
                            'Year': int(year),
                            'Value': value
                        })
            else:
                # No data available for the combination
                print(f"No data available for country: {country}, indicator: {indicator_name}")
        else:
            # Unexpected structure or invalid value
            print(f"Invalid or unexpected response for country: {country}, indicator: {indicator_name}. Response: {response_data}")

# Create a pandas DataFrame from the collected data
df = pd.DataFrame(data_results)

# Pivot the DataFrame to make it tidy
tidy_df = df.pivot_table(index=['Country', 'Year'], columns='Indicator', values='Value').reset_index()

# Remove the hierarchical name
tidy_df.columns.name = None

# Filter the DataFrame for the year 2019
filtered_df_2019 = tidy_df[tidy_df['Year'] == 2019]

# Display the filtered DataFrame
print(filtered_df_2019)


Invalid or unexpected response for country: BEN, indicator: Trading Across Borders. Response: [{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided parameter value is not valid'}]}]
Invalid or unexpected response for country: BEN, indicator: Protecting Minority Investors. Response: [{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided parameter value is not valid'}]}]
Invalid or unexpected response for country: BEN, indicator: Political Stability Index. Response: [{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided parameter value is not valid'}]}]
Invalid or unexpected response for country: BFA, indicator: Trading Across Borders. Response: [{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided parameter value is not valid'}]}]
Invalid or unexpected response for country: BFA, indicator: Protecting Minority Investors. Response: [{'message': [{'id': '120', 'key': 'Invalid value', 'value': 'The provided pa

In [41]:
tidy_df.head()

,Country,Year,Access to Electricity,Control of Corruption,Ease of Doing Business,Education Expenditure (% of GDP),Foreign Direct Investment (FDI) Inflows,GDP By Sector,GDP Growth Rate,GDP per Capita,...,Inflation Rate,Internet Penetration,Labor Force Participation Rate,Population Size,Poverty Headcount Ratio,Regulatory Quality,Starting a Business,Tariff Rates,Urban Population,Youth Unemployment Rate
0,BEN,2019,40.0,-0.329360,149.0,2.965201,1.516311,1.439071e+10,6.865687,1170.885995,...,0.732849,20.5000,62.671,12290444.0,NaN,-0.446330,8.5,9.86,47.861,3.781
1,BEN,2020,41.0,-0.039982,NaN,3.094415,1.109344,1.568674e+10,3.848792,1240.733155,...,3.022721,22.1206,62.076,12643123.0,NaN,-0.483432,NaN,9.86,48.415,4.240
2,BEN,2021,42.0,-0.170539,NaN,3.203484,1.956087,1.768762e+10,7.155452,1360.911474,...,1.733540,30.7419,61.106,12996895.0,12.7,-0.447988,NaN,10.89,48.972,4.207
3,BEN,2022,56.5,-0.089857,NaN,3.377859,2.157704,1.742542e+10,6.253245,1304.994797,...,1.350779,33.7602,62.399,13352864.0,NaN,-0.348365,NaN,NaN,49.534,3.913
4,BEN,2023,NaN,-0.049579,NaN,NaN,2.205282,1.967328e+10,6.352822,1434.662834,...,2.731025,NaN,62.709,13712828.0,NaN,-0.295836,NaN,NaN,50.100,3.888


In [ ]:
#exporting the data to csv
tidy_df.to_csv("data/indicators.csv", index=False)

In [ ]:
# Ignor for now

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import zipfile

# Step 1: Set up URLs and Paths
BASE_URL = "https://wits.worldbank.org"
DOWNLOAD_PAGE = "https://wits.worldbank.org/bulkdownload.aspx"
DOWNLOAD_FOLDER = "./wits_data"

# Create folder to store downloads if not exists
if not os.path.exists(DOWNLOAD_FOLDER):
    os.makedirs(DOWNLOAD_FOLDER)

# Step 2: Extract the links for bulk download from WITS
def get_bulk_download_links():
    response = requests.get(DOWNLOAD_PAGE)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract links to the zip files
    links = []
    for link in soup.find_all('a', href=True):
        if "bulkdownload" in link['href']:
            full_link = BASE_URL + link['href']
            links.append(full_link)
    
    return links

# Step 3: Download and Extract Files
def download_and_extract_links(links):
    for link in links:
        # Extract file name from URL
        file_name = link.split("/")[-1]
        zip_path = os.path.join(DOWNLOAD_FOLDER, file_name)
        
        # Download the zip file
        with requests.get(link, stream=True) as r:
            r.raise_for_status()
            with open(zip_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        
        # Extract the zip file contents
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DOWNLOAD_FOLDER)

# Step 4: Load and Process the CSV Data
def process_downloaded_data():
    # List all CSV files in the download folder
    csv_files = [f for f in os.listdir(DOWNLOAD_FOLDER) if f.endswith('.csv')]
    
    # Load all CSVs and concatenate them
    combined_data = pd.DataFrame()
    for csv_file in csv_files:
        file_path = os.path.join(DOWNLOAD_FOLDER, csv_file)
        df = pd.read_csv(file_path, encoding='latin1')  # Encoding may vary
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    
    # Perform further data cleaning and preprocessing
    # Example: Drop duplicate rows, clean missing values, etc.
    combined_data.drop_duplicates(inplace=True)
    combined_data.dropna(how='all', inplace=True)

    # Example: Filter for specific countries or indicators if necessary
    west_african_countries = ['Benin', 'Burkina Faso', 'Cabo Verde', 'Cote d\'Ivoire', 
                              'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Liberia',
                              'Mali', 'Mauritania', 'Niger', 'Nigeria', 'Senegal',
                              'Sierra Leone', 'Togo']
    
    combined_data = combined_data[combined_data['Country'].isin(west_african_countries)]
    
    return combined_data

# Step 5: Main Function
if __name__ == "__main__":
    # Step 2: Get bulk download links
    print("Fetching bulk download links...")
    links = get_bulk_download_links()
    print(f"Found {len(links)} links for download.")

    # Step 3: Download and Extract data
    print("Downloading and extracting data...")
    download_and_extract_links(links)
    
    # Step 4: Process the downloaded data
    print("Processing downloaded data...")
    combined_data = process_downloaded_data()

    # Save the processed data to a CSV file
    processed_file_path = os.path.join(DOWNLOAD_FOLDER, "combined_trade_indicators.csv")
    combined_data.to_csv(processed_file_path, index=False)

    print(f"Processed data saved to {processed_file_path}")


Fetching bulk download links...
Found 0 links for download.
Processing downloaded data...


KeyError: 'Country'

In [ ]:
#installing libraries
!pip install world_trade_data --upgrade

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): still running...
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for world_trade_data: filename=world_trade_data-0.1.1-py3-none-any.whl size=8857 sha256=e54c2d55e176c1af26225b4601479ce5fb98d90895525c9ceb06471a2df88687
  Stored in directory: c:\users\mgt\appdata\local\pip\cache\wheels\1b\0f\ba\a630deb44f697d86b66c36af82dfc3dabb3857c9c60785697a
Successfully built world_trade_data



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Data for table_1_tidy:
                       Indicator                Year       Value
0        Population (000, 2021)                                
1  Pop. density (per km2, 2021)                                
2            Surface area (km2)                                
3       Sex ratio (m per 100 f)                                
4        Population (000, 2021)  Sub-Saharan Africa    412 460a
5  Pop. density (per km2, 2021)  Sub-Saharan Africa         68a
6            Surface area (km2)  Sub-Saharan Africa  6 064 060b
7       Sex ratio (m per 100 f)  Sub-Saharan Africa    101.4a,c 

Data for table_2_tidy:
                                              Indicator  Year  \
0    GDP: Gross domestic product (million current US$)  2010   
1       GDP growth rate (annual %, const. 2015 prices)  2010   
2                         GDP per capita (current US$)  2010   
3           Employment in agricultured (% of employed)  2010   
4              Employment in industryd (% of employed)

In [ ]:
########### Ignore for now #############
import urllib.request, json

try:
    url = "https://comtradeapi.un.org/data/v1/getDa/C/A/S4"

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    print(response.getcode())
    print(response.read())
except Exception as e:
    print(e)
####################################